# Training of Models

In [ ]:
#upgrade pip and protobuf
!pip install --upgrade pip
!pip install --upgrade protobuf

In [ ]:
#install tensorflow version 1.15 and numpy
%tensorflow_version 1.15
import tensorflow as tf
print(tf.__version__)

!pip install numpy

In [ ]:
#check your GPU status
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
#mount Google Drive Folder
from google.colab import drive
drive.mount('/content/drive')

# change to working tensorflow directory on the drive
%cd '/content/drive/My Drive/models/'

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
%cd /content/drive/My Drive/models/research/
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/models/research/:/content/drive/My Drive/models/research/slim'

!python setup.py build
!python setup.py install

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

In [ ]:
#performing the training of models
!pip install tf_slim
%cd /content/drive/My Drive/models/research/Object_Detection
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/models/research/:/content/drive/My Drive/models/research/slim'


!python legacy/train.py --train_dir=training/ --pipeline_config_path=training/faster_rcnn_inception_resnet_v2_atrous_coco.config --logtostderr

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir 'training'

In [ ]:
#the XXXX is dependent on the number of steps in the config files from the pretrained model
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_resnet_v2_atrous_coco.config --trained_checkpoint_prefix training/model.ckpt-XXXX --output_directory faster_new_graph

In [ ]:
#converting the model graph to a zip folder before downloading
!zip -r faster_model_graph.zip faster_new_graph